## Import

In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

## Definition of classes

In [2]:
class Milkrun:
    def __init__(self, to):
        self.TOs_covered = [to]
        self.origins = {to.origin}
        self.destinations = {to.destinations}
        self.type = "neither"

    def number_of_tos(self):
        return len(self.TOs_covered)

    def add_to(self, to):
        self.origins.push(to.origin)
        self.destinations.push(to.origin)
        if len(self.origins) > 1 and len(self.destinations) > 1:
            self.origins.pop()
            self.destinations.pop()
            return False
        if len(self.origins) > 1:
            self.type = "inbound"
        if len(self.destinations) > 1:
            self.type = "outbound"
        self.TOs_covered.append(to)

    def total_weight(self):
        weight = 0
        for to in self.TOs_covered:
            weight += to.weight
        return weight

    def total_length(self):
        length = 0
        for to in self.TOs_covered:
            length += to.length
        return length

    def total_volume(self):
        volume = 0
        for to in self.TOs_covered:
            volume += to.volume
        return volume

    def pop(self):
        self.TOs_covered.pop()

class MoT:
    def __init__(self, name, avg_speed, payload, length, width, height):
        self.name = name
        self.avg_speed = avg_speed
        self.max_payload = payload
        self.max_length = length
        self.max_width = width
        self.max_height = height
        self.rem_length = length
        self.rem_width = width
        self.rem_height = height
        self.rem_payload = payload
        self.max_vol = length * width * height
        self.TOs_loaded = []

    def load_truck(self, TO):
        if self._check_fits():
            pass
        pass

    def unload_truck(self, TO):
        pass

    def empty_truck(self, TO):
        pass

    def _check_fits(self, TO):
        fits = True
        pass


# Transport Order Class
class Order:
    def __init__(self, order_num, origin, destination, weight, length, volume):
        self.order_num = order_num
        self.origin = origin
        self.destination = destination
        self.weight = weight
        self.length = length
        self.volume = volume

## Helper Functions 

In [3]:
def get_to_list():
    """Reads Transport Orders from File, Instantiates and Order for each one, and returns a list of these Orders"""

    # Read in Transport Orders from File
    tos = pd.read_csv("./Data/TransportOrders.csv")

    # Clean out units and convert to numeric
    tos['Weight'] = pd.to_numeric(tos['Weight'].str.rstrip(' kg').str.replace(',', ''))
    tos['Loading Meters'] = pd.to_numeric(tos['Loading Meters'].str.split(expand=True)[0])
    tos['Volume'] = pd.to_numeric(tos['Volume'].str.split(expand=True)[0])

    # Create a list of Instances of the Order class
    to_list = []
    for index, row in tos.iterrows():
        to_list.append(Order(row['Transport Order'],
                             row['Origin Index'],
                             row['Destination Index'],
                             row['Weight'],
                             row['Loading Meters'],
                             row['Volume'])
                       )

    return to_list

# LTL Tariff - no distance
def get_tariff(weight):
    tariff_levels = pd.Series({1.79: 0,
                               1.70: 201,
                               1.62: 501,
                               1.53: 1001,
                               1.46: 1501,
                               1.38: 2001,
                               1.32: 3001,
                               1.25: 4001,
                               1.19: 5001,
                               1.13: 7501,
                               1.07: 10001})
    cost = tariff_levels[tariff_levels <= weight].index[-1]

    tariff = cost * weight

    if tariff >= 250:
        return tariff
    else:
        return 250


# LTL Tariff - with distance
def get_tariff_dist(weight, dist):
    tariff_levels = pd.read_csv("./Data/LTLTariff.csv", index_col=0)
    tariff_levels.columns = tariff_levels.columns.astype(float)

    x = tariff_levels.columns[tariff_levels.columns <= dist][-1]
    y = tariff_levels.index[tariff_levels.index <= weight][-1]

    tariff = tariff_levels.loc[y, x]

    return tariff


# FTL Tariff
def get_tariff_ftl(dist):
    transport_cost = 50
    transport_time = 0  # will need to be updated if this comes into play
    distance_rate = 0.2  # Euro/km
    return transport_cost + (dist * distance_rate)


# Milk Run Tariff
def get_tariff_milk(dist, num_stops):
    transport_cost = 100
    transport_time = 0  # will need to be updated if this comes into play
    distance_rate = 0.6
    stop_cost = 40
    return transport_cost + distance_rate * dist + num_stops * stop_cost


In [9]:
data = {'distance_matrix': np.array([
    [0, 170, 210, 2219, 1444, 2428],  # Nuremberg
    [170, 0, 243, 2253, 1369, 2354],  # Munich
    [210, 243, 0, 2042, 1267, 2250],  # Stuttgart
    [2219, 2253, 2042, 0, 1127, 579],  # Supplier Porto
    [1444, 1369, 1267, 1127, 0, 996]  # Supplier Barcelona
])}

TO_list = get_to_list()

data['pickups_deliveries'] = []
for i in TO_list:
    data['pickups_deliveries'].append([i.origin, i.destination])

model = pyo.ConcreteModel()

# Sets
tariffs = list(range(1, 1 + len(TO_list)))

# Variables
model.x = pyo.Var(
    tariffs, [1, 2],
    within=pyo.Binary,
    doc="TO utilizes LTL if (i,1) is 1 or FTL if (i,2) is 1"
)


# Constraints


# Objective

def cost_func(mdl):
    cost = 0
    for i in range(1, 1 + len(TO_list)):
        cost += mdl.x[i, 1] * get_tariff_dist(data['distance_matrix'][tuple(data['pickups_deliveries'][i - 1])],
                                              TO_list[i - 1].weight) + mdl.x[i, 2] * get_tariff_ftl(
            data['distance_matrix'][tuple(data['pickups_deliveries'][i - 1])])
    return cost


def cost_to(to, ltl, ftl):
    return ltl * get_tariff_dist(data['distance_matrix'][tuple(data['pickups_deliveries'][to - 1])],
                                 TO_list[to - 1].weight) + ftl * get_tariff_ftl(
        data['distance_matrix'][tuple(data['pickups_deliveries'][to - 1])])


model.Cost = pyo.Objective(rule=cost_func, sense=pyo.minimize)

model.Constraint1 = pyo.ConstraintList()
for i in range(1, 1 + len(TO_list)):
    model.Constraint1.add(expr=model.x[i, 1] + model.x[i, 2] >= 1)

# Solve

instance = model.create_instance()
results = SolverFactory('glpk').solve(instance)
results.write()
instance.display()
instance.solutions.load_from(results)

a = MoT('Standard 25to', 50, 25000, 13.6, 2.5, 2.48)
b = MoT('MEGA', 70, 25000, 13.62, 2.48, 3)
c = MoT('PICKUP 3.5t', 60, 3500, 6.4, 2.5, 2.5)

tos = pd.DataFrame({"transportOrder": [to.order_num for to in TO_list], "origin": [to.origin for to in TO_list],
                    "destination": [to.destination for to in TO_list], "weight": [to.weight for to in TO_list],
                    "length": [to.length for to in TO_list], "volume": [to.volume for to in TO_list],
                    "cost": [cost_to(to, instance.x[(to, 1)].value, instance.x[(to, 2)].value) for to in
                             range(1, 1 + len(TO_list))], "milkrun": False, "considered": False})
tos = tos.sort_values(by=["cost"], ascending=False)
milkrun_list = []


def find_to(to_name):
    for to in TO_list:
        if to.order_num == to_name:
            return to


while len(tos[~tos["considered"]].index) > 0:
    to_consider = tos[~tos["considered"] & ~tos["milkrun"]][:1]
    tos.loc[tos["transportOrder"].str.match(to_consider["transportOrder"][0]), "considered"] = True
    new_milkrun = Milkrun(find_to(to_consider["transportOrder"][0]))
    while True:
        to_add = tos.query('~milkrun and ~considered and (origin == ' + str(to_consider["origin"][0])
                           + ' or destination == '
                           + str(to_consider["destination"][0])
                           + ') and weight < '
                           + str(b.max_payload - new_milkrun.total_weight())
                           + ' and length < '
                           + str(b.max_length - new_milkrun.total_length())
                           + ' and volume <'
                           + str(b.max_vol - new_milkrun.total_volume()))

    model; returning a clone of the current model instance. (called from
    /Users/epanza/opt/anaconda3/envs/Urban/lib/python3.8/site-
    packages/pyomo/core/base/PyomoModel.py:681)
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19210.12
  Upper bound: 19210.12
  Number of objectives: 1
  Number of constraints: 44
  Number of variables: 87
  Number of nonzeros: 87
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number 

AttributeError: 'Order' object has no attribute 'destinations'